# Imputation Research Project <img src="https://chroniclesofai.com/content/images/2021/05/file-20201210-18-elk4m.jpg" alt="Alt text image not displaying" width="360" align="right" />
## Notebook 3.0: Autoencoder Model

**Author:** Chike Odenigbo

**Date:** November 25th, 2022

**Notebook Structure:**

* 1.0 Preprocessing

* **1.1 Exploratory Data Analysis**

* 1.2 Masking

* 2.* Models


Water Sugar lutein_zeaxanthin Alcohol

In [1]:
import pandas as pd
import os
from src.visualization.visualize import histogram, box_plot, bar_plot
from pathlib import Path
from notebook_config import ROOT_DIR  # setup.py file changed the root of the project so it is set in the config file

ROOT_DIR = ROOT_DIR.as_posix()  # convert root path to windows readable path (i.e. change backslash to forward slash)
from joblib import load
import tensorflow as tf
from keras import backend as K
import keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import pickle

In [2]:
notebook_nm = "3.0-autoencoder"
fig_dir = f"{ROOT_DIR}/reports/figures/"
model_dir = f"{ROOT_DIR}/models/autoencoders/"
scaler_dir = f'{ROOT_DIR}/models/scalers/'
output_prefix = notebook_nm

In [3]:
pd.set_option('display.max_columns', None)

# Ground Truth Included
water_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water.csv')
sugars_df = pd.read_csv(f'{ROOT_DIR}/data/processed/sugars.csv')
lutein_df = pd.read_csv(f'{ROOT_DIR}/data/processed/lutein.csv')

# Scaled Data without Ground Truth to prevent Data Leakage, rows are included with NaN
water_mcar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water_mcar_scaled.csv', index_col = 0)
water_mar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water_mar_scaled.csv', index_col = 0)

lutein_mar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/lutein_mar_scaled.csv', index_col = 0)
lutein_mcar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/utein_mcar_scaled.csv', index_col = 0)

sugars_mar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/sugars_mar_scaled.csv', index_col = 0)
sugars_mcar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/sugars_mcar_scaled.csv', index_col = 0)

water_mcar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water_mcar_scaled.csv', index_col = 0)
water_mar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water_mar_scaled.csv', index_col = 0)

# Scalers to return back to origin scale for model evaluation
scaler_lutein_mar = load(f'{scaler_dir}/scaler_lutein_mar.joblib')
scaler_lutein_mcar = load(f'{scaler_dir}/scaler_lutein_mcar.joblib')

scaler_sugars_mar = load(f'{scaler_dir}/scaler_sugars_mar.joblib')
scaler_sugars_mcar = load(f'{scaler_dir}/scaler_sugars_mcar.joblib')

scaler_water_mcar = load(f'{scaler_dir}/scaler_water_mcar.joblib')
scaler_water_mar = load(f'{scaler_dir}/scaler_water_mar.joblib')

In [4]:
def scale_ground_truth(scaler,raw_df,target_col,non_target_na_col):
    raw_filtered_df = raw_df[raw_df[target_col].isnull()].drop(['name',target_col, non_target_na_col], axis = 'columns')
    scaled_ground_truth_df = pd.DataFrame(scaler.transform(raw_filtered_df),columns = raw_filtered_df.columns)
    return scaled_ground_truth_df

def train_test_split_scaled(scaled_df,drop_cols = ['name','dataset_type']):
    train_df = scaled_df[scaled_df.dataset_type == 'training'].drop(drop_cols, axis = 'columns')
    val_df = scaled_df[scaled_df.dataset_type == 'validation'].drop(drop_cols, axis = 'columns')
    return train_df, val_df

def autoencoder(data, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.0001):
    enc_input = keras.Input(shape = (input_shape,),name = 'input_data')
    latent_layer = keras.layers.Dense(latent_shape, activation = 'relu')(enc_input)
    encoder = keras.Model(enc_input,latent_layer, name = "encoder")
    dec_output = keras.layers.Dense(input_shape)(latent_layer)
    opt = keras.optimizers.Adam(lr = lr)
    auto_encoder = keras.Model(enc_input,dec_output, name="auto_encoder")
    print(auto_encoder.summary())
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    auto_encoder.compile(opt, loss = "mse")
    history = auto_encoder.fit(data, data, epochs = 1000, validation_split = 0.1, callbacks=[callback])
    return auto_encoder, history

def save_autoencoder(model,model_name,path = model_dir):
    model.save(f'{path}/{model_name}.h5')
    with open(f'{path}/{model_name}_history.pkl', 'wb') as file_pi:
        pickle.dump(model.history, file_pi)

In [5]:
water_mcar_train_df, water_mcar_val_df = train_test_split_scaled(water_mcar_scaled_df,drop_cols = ['name','dataset_type'])
water_mar_train_df, water_mar_val_df = train_test_split_scaled(water_mar_scaled_df,drop_cols = ['name','dataset_type'])

sugars_mcar_train_df, sugars_mcar_val_df = train_test_split_scaled(sugars_mcar_scaled_df,drop_cols = ['name','dataset_type'])
sugars_mar_train_df, sugars_mar_val_df = train_test_split_scaled(sugars_mar_scaled_df,drop_cols = ['name','dataset_type'])

lutein_mcar_train_df, lutein_mcar_val_df = train_test_split_scaled(lutein_mcar_scaled_df,drop_cols = ['name','dataset_type'])
lutein_mar_train_df, lutein_mar_val_df = train_test_split_scaled(lutein_mar_scaled_df,drop_cols = ['name','dataset_type'])


## Train Models

In [6]:
sugars_mcar_model, sugars_mcar_history = autoencoder(sugars_mcar_train_df, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.001)

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense (Dense)               (None, 40)                2920      
                                                                 
 dense_1 (Dense)             (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


None
Epoch 1/1000
140/140 [==============================] - 2s 11ms/step - loss: 0.9123 - val_loss: 0.1546
Epoch 2/1000
140/140 [==============================] - 1s 10ms/step - loss: 0.5648 - val_loss: 0.1112
Epoch 3/1000
140/140 [==============================] - 1s 6ms/step - loss: 0.4339 - val_loss: 0.0971
Epoch 4/1000
140/140 [==============================] - 1s 6ms/step - loss: 0.3500 - val_loss: 0.0839
Epoch 5/1000
140/140 [==============================] - 1s 7ms/step - loss: 0.2925 - val_loss: 0.0720
Epoch 6/1000
140/140 [==============================] - 1s 6ms/step - loss: 0.2490 - val_loss: 0.0626
Epoch 7/1000
140/140 [==============================] - 1s 9ms/step - loss: 0.2154 - val_loss: 0.0570
Epoch 8/1000
140/140 [==============================] - 1s 9ms/step - loss: 0.1877 - val_loss: 0.0510
Epoch 9/1000
140/140 [==============================] - 1s 8ms/step - loss: 0.1660 - val_loss: 0.0484
Epoch 10/1000
140/140 [==============================] - 1s 9ms/step - loss

In [7]:
sugars_mar_model, sugars_mar_history = autoencoder(sugars_mar_train_df, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.001)

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense_2 (Dense)             (None, 40)                2920      
                                                                 
 dense_3 (Dense)             (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/1000
183/183 [==============================] - 3s 8ms/step - loss: 0.5843 - val_loss: 0.1248
Epoch 2/1000
183/183 [==============================] - 1s 6ms/step - loss: 0.3230 - val_loss: 0.0870
Epoch 3/1000
183/183 [==============================] - 1s 7ms/step - loss: 0.2404 - val_loss: 0.0685
Epoch 4/1000
183/183 [==============================] - 1s 5ms/step - loss: 0.1897 - val_loss: 0.0579
Epoch 5/1000
183/183 [==============================] - 1s 6ms/step - loss: 0.1562 - val_loss: 0.0480
Epoch 6/1000
183/183 [==============================] - 1s 7ms/step - loss: 0.1318 - val_loss: 0.0441
Epoch 7/1000
183/183 [==============================] - 1s 7ms/step - loss: 0.1138 - val_loss: 0.0416
Epoch 8/1000
183/183 [==============================] - 1s 5ms/step - loss: 0.0991 - val_loss: 0.0382
Epoch 9/1000
183/183 [==============================] - 1s 6ms/step - loss: 0.0870 - val_loss: 0.0378
Epoch 10/1000
183/183 [==============================] - 1s 6ms/step - loss: 0.076

In [8]:
water_mar_model, water_mar_history = autoencoder(water_mar_train_df, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.001)

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense_4 (Dense)             (None, 40)                2920      
                                                                 
 dense_5 (Dense)             (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/1000
183/183 [==============================] - 2s 7ms/step - loss: 0.5912 - val_loss: 0.1174
Epoch 2/1000
183/183 [==============================] - 1s 6ms/step - loss: 0.3338 - val_loss: 0.0848
Epoch 3/1000
183/183 [==============================] - 1s 8ms/step - loss: 0.2461 - val_loss: 0.0684
Epoch 4/1000
183/183 [==============================] - 1s 7ms/step - loss: 0.1928 - val_loss: 0.0507
Epoch 5/1000
183/183 [==============================] - 1s 7ms/step - loss: 0.1585 - val_loss: 0.0428
Epoch 6/1000
183/183 [==============================] - 1s 6ms/step - loss: 0.1338 - val_loss: 0.0384
Epoch 7/1000
183/183 [==============================] - 1s 7ms/step - loss: 0.1154 - val_loss: 0.0351
Epoch 8/1000
183/183 [==============================] - 1s 5ms/step - loss: 0.1011 - val_loss: 0.0321
Epoch 9/1000
183/183 [==============================] - 1s 7ms/step - loss: 0.0889 - val_loss: 0.0298
Epoch 10/1000
183/183 [==============================] - 1s 7ms/step - loss: 0.078

In [9]:
water_mcar_model, water_mcar_history = autoencoder(water_mcar_train_df, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.001)

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense_6 (Dense)             (None, 40)                2920      
                                                                 
 dense_7 (Dense)             (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/1000
161/161 [==============================] - 2s 8ms/step - loss: 0.9285 - val_loss: 0.2194
Epoch 2/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.5629 - val_loss: 0.1610
Epoch 3/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.4232 - val_loss: 0.1427
Epoch 4/1000
161/161 [==============================] - 1s 7ms/step - loss: 0.3386 - val_loss: 0.1241
Epoch 5/1000
161/161 [==============================] - 1s 8ms/step - loss: 0.2821 - val_loss: 0.1090
Epoch 6/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.2414 - val_loss: 0.0992
Epoch 7/1000
161/161 [==============================] - 1s 8ms/step - loss: 0.2072 - val_loss: 0.0904
Epoch 8/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.1804 - val_loss: 0.0844
Epoch 9/1000
161/161 [==============================] - 1s 8ms/step - loss: 0.1591 - val_loss: 0.0786
Epoch 10/1000
161/161 [==============================] - 1s 7ms/step - loss: 0.141

In [10]:
lutein_mar_model, lutein_mar_history = autoencoder(lutein_mar_train_df, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.001)

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense_8 (Dense)             (None, 40)                2920      
                                                                 
 dense_9 (Dense)             (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/1000
162/162 [==============================] - 2s 8ms/step - loss: 0.6565 - val_loss: 0.1269
Epoch 2/1000
162/162 [==============================] - 1s 6ms/step - loss: 0.3778 - val_loss: 0.0953
Epoch 3/1000
162/162 [==============================] - 1s 8ms/step - loss: 0.2831 - val_loss: 0.0797
Epoch 4/1000
162/162 [==============================] - 1s 5ms/step - loss: 0.2241 - val_loss: 0.0678
Epoch 5/1000
162/162 [==============================] - 1s 8ms/step - loss: 0.1840 - val_loss: 0.0592
Epoch 6/1000
162/162 [==============================] - 1s 6ms/step - loss: 0.1542 - val_loss: 0.0522
Epoch 7/1000
162/162 [==============================] - 1s 5ms/step - loss: 0.1318 - val_loss: 0.0472
Epoch 8/1000
162/162 [==============================] - 1s 6ms/step - loss: 0.1147 - val_loss: 0.0436
Epoch 9/1000
162/162 [==============================] - 1s 5ms/step - loss: 0.1004 - val_loss: 0.0397
Epoch 10/1000
162/162 [==============================] - 1s 6ms/step - loss: 0.088

In [11]:
lutein_mcar_model, lutein_mcar_history = autoencoder(lutein_mcar_train_df, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.001)

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense_10 (Dense)            (None, 40)                2920      
                                                                 
 dense_11 (Dense)            (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/1000
140/140 [==============================] - 2s 9ms/step - loss: 0.9269 - val_loss: 0.1555
Epoch 2/1000
140/140 [==============================] - 1s 6ms/step - loss: 0.5618 - val_loss: 0.1064
Epoch 3/1000
140/140 [==============================] - 1s 7ms/step - loss: 0.4354 - val_loss: 0.0839
Epoch 4/1000
140/140 [==============================] - 1s 8ms/step - loss: 0.3523 - val_loss: 0.0735
Epoch 5/1000
140/140 [==============================] - 1s 7ms/step - loss: 0.2933 - val_loss: 0.0614
Epoch 6/1000
140/140 [==============================] - 1s 8ms/step - loss: 0.2518 - val_loss: 0.0592
Epoch 7/1000
140/140 [==============================] - 1s 6ms/step - loss: 0.2205 - val_loss: 0.0539
Epoch 8/1000
140/140 [==============================] - 1s 6ms/step - loss: 0.1951 - val_loss: 0.0504
Epoch 9/1000
140/140 [==============================] - 1s 7ms/step - loss: 0.1736 - val_loss: 0.0547
Epoch 10/1000
140/140 [==============================] - 1s 7ms/step - loss: 0.156

In [12]:
save_autoencoder(model = sugars_mcar_model, model_name = 'sugars_mcar_model',path = model_dir)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 17:04:08         1872
metadata.json                                  2022-11-28 17:04:08           64
variables.h5                                   2022-11-28 17:04:08        36504


In [13]:
save_autoencoder(model = sugars_mar_model, model_name = 'sugars_mar_model',path = model_dir)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 17:04:08         1878
metadata.json                                  2022-11-28 17:04:08           64
variables.h5                                   2022-11-28 17:04:08        36504


In [14]:
save_autoencoder(model = water_mar_model, model_name = 'water_mar_model',path = model_dir)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 17:04:08         1878
metadata.json                                  2022-11-28 17:04:08           64
variables.h5                                   2022-11-28 17:04:08        36504


In [15]:
save_autoencoder(model = water_mcar_model, model_name = 'water_mcar_model',path = model_dir)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 17:04:08         1878
metadata.json                                  2022-11-28 17:04:08           64
variables.h5                                   2022-11-28 17:04:08        36504


In [16]:
save_autoencoder(model = lutein_mar_model, model_name = 'lutein_mar_model',path = model_dir)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 17:04:09         1878
metadata.json                                  2022-11-28 17:04:09           64
variables.h5                                   2022-11-28 17:04:09        36504


In [17]:
save_autoencoder(model = lutein_mcar_model, model_name = 'lutein_mcar_model',path = model_dir)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 17:04:09         1884
metadata.json                                  2022-11-28 17:04:09           64
variables.h5                                   2022-11-28 17:04:09        36504


In [18]:
max_water_mcar = water_mcar_train_df.water_mcar.max()
min_water_mcar = water_mcar_train_df.water_mcar.min()

In [19]:
water_mcar_test_df['water_mcar'] = 0#np.random.uniform(min_water_mcar, max_water_mcar, size=len(water_mcar_test_df))

NameError: name 'water_mcar_test_df' is not defined

In [ ]:
water_mcar_test_df

In [ ]:
water_mcar_pred = pd.DataFrame(auto_encoder.predict(water_mcar_test_df), columns = water_mcar_test_df.columns)

In [ ]:
water_mcar_true = pd.DataFrame(scaler_water_mcar.transform(water_df[water_df.water_mcar.isnull()].drop(['name','water_mcar', 'water_mar'], axis = 'columns')),columns = water_df.drop(['name','water_mcar', 'water_mar'], axis = 'columns').columns)# a = np.array(x) # your x
# b = np.array(y) # your y
# mses = ((a-b)**2).mean(axis=1)

In [ ]:
from sklearn.metrics import mean_squared_error
pd.DataFrame({'columns':water_mcar_true.columns,
              'mse':list(mean_squared_error(water_mcar_true, water_mcar_pred, multioutput='raw_values'))}).nlargest(10,'mse')